In [1]:
import os
import importlib
import numpy as np
import yaml
from tqdm.auto import tqdm
import seesaw
import ray 
import seesaw.seesaw_bench
import seesaw.figures
import seesaw.dataset_manager
import seesaw.dataset

importlib.reload(seesaw.metrics)
importlib.reload(seesaw.figures)
importlib.reload(seesaw.seesaw_bench)

import seesaw.figure_utils
importlib.reload(seesaw.figure_utils)
from seesaw.figure_utils import *
from seesaw.seesaw_bench import *
from seesaw.figures import *
from plotnine import *
from seesaw.dataset_manager import GlobalDataManager


In [2]:
#ray.init('auto', namespace='seesaw')

In [3]:
import glob

In [16]:
results_paths = [    
    # ('', '//home/gridsan/omoll/test_new_trainer/bench_DHeCZhAeux/'),    
    # ('', '//home/gridsan/omoll/test_new_trainer/bench_LAaCMfitvl/'), #'baseline'
    # ('', '//home/gridsan/omoll/test_new_trainer/bench_jYTXlGTLdu/'), # rocchio hyperparams
    ('', './bench_NrMYlLXfJo/'), # seesaw hyperparams 1
    ('', './bench_AEZJiQmHaT/'), # seesaw hyperparams 2
    ('', './bench_jYTXlGTLdu/'), # rocchio hyperparams
    
]

## are the results between current version and previous version really identical? unlikely...
## next steps: 
# 1. check something changed for some dataset before and after.
# if yes, then nothing to do.
# or check if the lambda is perhaps too high.

#all_summs = [checkpoint]
all_summs = []
for (i,(nm, bench_folder)) in enumerate(results_paths):
    summss = get_all_session_summaries(bench_folder, force_recompute=False, parallel=False)
    summss = summss.assign(bench_folder=bench_folder, bench_id=bench_folder.rstrip('/').rsplit('/')[-1])
    summss = summss.assign(run_id = summss.bench_id + ('_' + summss.sample_id).fillna(''), timestamp=get_timestamp(bench_folder))
    
    
    if nm != '':
        summss = summss.assign(name=summss.name + '_' + nm)
        
    all_summs.append(summss)
    
summs = pd.concat(all_summs, ignore_index=True).reset_index(drop=True)
#summs.to_parquet('/home/gridsan/omoll/concat_summs_checkpoint_Mar1.parquet')

In [38]:
### table with results vs. baselines.
### table with hyperparam results for both rocchio, seesaw.

In [17]:
bdd_ignore = ['other vehicle', 'side traffic light', 
              'green traffic light', 'yellow traffic light', 'red traffic light', 
              'dawn or dusk', 'daytime', 'night']

stats = summs.assign(**post_process(summs))
stats = stats.assign(session_params_dict=stats.session_params.map(lambda p : json.loads(p)))
stats = stats.assign(start_policy=stats.session_params_dict.map(lambda x : x['start_policy']))


In [12]:
#stats.session_params_dict.iloc[-1]

In [18]:
ps = (stats[['session_params_dict']]
    .apply(apply_funs(get_active_search_params2, 
                      get_matrix_params, 
                      get_multi_reg_params,
                        get_rocchio_params), axis=1, result_type='expand'))
stats = stats.assign(**ps)

stats = stats[stats.category.map(lambda x : x not in bdd_ignore and not x.endswith('weather') and not x.endswith('scene'))]

subset = stats[stats.index_name.isin(['multiscalecoarse', 'multiscalemed']) 
                    &  stats.reg_norm_lambda.isin([np.nan, 100.]) 
                    & stats.variant.isin(['baseline', 'multi_reg', 'active_search', 'rocchio_update'])] 

In [26]:
old_eval_data = pd.read_parquet('../figures/eval_data.parquet')
baseline_df = old_eval_data[old_eval_data.full_hash.eq('2501cb92_multiscalecoarse')]

#subset = pd.read_parquet('./rocchio_hyperparam.parquet')
subset = pd.concat([stats, baseline_df])

In [27]:
def add_hard_queries(eval_data):
    baseline_df = eval_data[eval_data.full_hash.eq('2501cb92_multiscalecoarse')]
    hard_baseline_df = baseline_df[baseline_df.average_precision < .5]
    hard_queries_df = hard_baseline_df[['dataset', 'category']].sort_values(['dataset', 'category']).reset_index(drop=True)
    eval_hard_df = eval_data.merge(hard_queries_df, left_on=('dataset', 'category'), right_on=('dataset', 'category'), how='right')

        
    eval_hard_df = eval_hard_df.assign(query_group='hard subset')
    eval_df = eval_data.assign(query_group='all queries')
    
    full_df = pd.concat([eval_df, eval_hard_df], ignore_index=True).reset_index(drop=True)
    full_df = full_df.assign(**{'query group':pd.Categorical(full_df.query_group, categories=['all queries', 'hard subset'], ordered=True)})
    
    return full_df

In [28]:
subset = add_hard_queries(subset)

In [29]:
subset.query_group.unique()

array(['all queries', 'hard subset'], dtype=object)

In [30]:
base_cols =  ['param_hash', 'index_name',  'timestamp', 'dataset', 'category','query_group']
detail_cols = ['full_hash', 'start_policy', 'variant', 'calibration', 'reward_horizon', 'pos_weight', 'reg_data_lambda', 'reg_query_lambda', 'reg_norm_lambda',
                          'rocchio_beta', 'rocchio_gamma']
metric_cols = ['reciprocal_rank', 'ndcg_score', 'average_precision']

In [31]:
eval_data = subset[base_cols + detail_cols + metric_cols]

In [45]:
s = display_means(eval_data,  metric='average_precision', other_cols=['query_group', 'index_name', 'variant',   'reg_query_lambda', 'reg_data_lambda', 'reg_norm_lambda', 'rocchio_beta', 'rocchio_gamma'], return_totals=True)

dataset,query_group,index_name,variant,reg_query_lambda,reg_data_lambda,reg_norm_lambda,rocchio_beta,rocchio_gamma,param_hash,timestamp,bdd,coco,lvis,objectnet,average
0,all queries,multiscalecoarse,baseline,nan,nan,nan,nan,nan,2501cb92,2023-03-15 17:42:10,0.74,0.90,0.63,0.64,0.72
1,all queries,multiscalemed,multi_reg,3.00,300.00,100.00,nan,nan,c8024c49,2023-07-16 16:42:57.231526,0.78,0.96,0.76,0.68,0.80
2,all queries,multiscalemed,multi_reg,3.00,1000.00,100.00,nan,nan,df383e32,2023-07-16 16:42:57.231526,0.77,0.97,0.77,0.68,0.80
3,all queries,multiscalemed,multi_reg,3.00,3000.00,100.00,nan,nan,d9b80fa6,2023-07-16 16:42:57.231526,0.77,0.96,0.76,0.63,0.78
4,all queries,multiscalemed,multi_reg,10.00,300.00,100.00,nan,nan,bb5e41c5,2023-07-16 16:42:57.231526,0.78,0.96,0.75,0.69,0.80
5,all queries,multiscalemed,multi_reg,10.00,1000.00,30.00,nan,nan,937e2c72,2023-07-16 19:06:01.214854,0.79,0.96,0.76,0.70,0.80
6,all queries,multiscalemed,multi_reg,10.00,1000.00,100.00,nan,nan,583a1edc,2023-07-16 16:42:57.231526,0.79,0.96,0.76,0.70,0.80
7,all queries,multiscalemed,multi_reg,10.00,1000.00,300.00,nan,nan,2148b502,2023-07-16 19:06:01.214854,0.79,0.96,0.76,0.70,0.80
8,all queries,multiscalemed,multi_reg,10.00,3000.00,100.00,nan,nan,310522e7,2023-07-16 16:42:57.231526,0.79,0.97,0.77,0.69,0.80
9,all queries,multiscalemed,multi_reg,30.00,300.00,100.00,nan,nan,bf8a0dc5,2023-07-16 16:42:57.231526,0.77,0.96,0.73,0.68,0.79


In [46]:
hptable = s[s.query_group == 'all queries']

In [47]:
tabs = hptable.drop([ 'index_name', 'timestamp', 'param_hash'], axis=1)
tabs = tabs[tabs.variant.isin(['multi_reg', 'rocchio_update'])]

for (k,),gp in  tabs.groupby(['variant']):
    gp = gp.dropna(axis=1)
    gp = gp.drop(['variant', 'query_group'], axis=1)
    gp = gp.rename({'reg_norm_lambda':'$\\lambda$',
                    'reg_query_lambda':'$\\lambda_q$',
                    'reg_data_lambda':'$\\lambda_D$',   
                    'rocchio_beta':'$\\beta$',
                   'rocchio_gamma':'$\\gamma$',
                   'lvis':'LVIS',
                    'objectnet':'ObjNet',
                    'average':'Avg.',
                    'bdd':'BDD',
                    'coco':'COCO',
                   }, axis=1)

#    gp.assign(variant=gp.variant.map(lambda v : {'multi_reg':'this work', 'rocchio_update':'Rocchio'}.get(v,v))
    str = (gp.style
               .hide(axis=0)
               .format(precision=2)
               .highlight_max(axis=0, subset=['BDD', 'COCO', 'LVIS', 'ObjNet', 'Avg.'], props='bfseries: ;')
               .to_latex(label=f'tab:hyperparam_{k}',hrules=True,
                         clines='all;index', caption=f'sensitivity to hyperparamters for method:{k}'))
    print(str)

\begin{table}
\caption{sensitivity to hyperparamters for method:multi_reg}
\label{tab:hyperparam_multi_reg}
\begin{tabular}{rrrrrrrr}
\toprule
$\lambda_q$ & $\lambda_D$ & $\lambda$ & BDD & COCO & LVIS & ObjNet & Avg. \\
\midrule
3.00 & 300.00 & 100.00 & 0.78 & 0.96 & 0.76 & 0.68 & 0.80 \\
3.00 & 1000.00 & 100.00 & 0.77 & 0.97 & \bfseries 0.77 & 0.68 & 0.80 \\
3.00 & 3000.00 & 100.00 & 0.77 & 0.96 & 0.76 & 0.63 & 0.78 \\
10.00 & 300.00 & 100.00 & 0.78 & 0.96 & 0.75 & 0.69 & 0.80 \\
10.00 & 1000.00 & 30.00 & 0.79 & 0.96 & 0.76 & 0.70 & 0.80 \\
10.00 & 1000.00 & 100.00 & \bfseries 0.79 & 0.96 & 0.76 & \bfseries 0.70 & \bfseries 0.80 \\
10.00 & 1000.00 & 300.00 & 0.79 & 0.96 & 0.76 & 0.70 & 0.80 \\
10.00 & 3000.00 & 100.00 & 0.79 & \bfseries 0.97 & 0.77 & 0.69 & 0.80 \\
30.00 & 300.00 & 100.00 & 0.77 & 0.96 & 0.73 & 0.68 & 0.79 \\
30.00 & 1000.00 & 100.00 & 0.77 & 0.96 & 0.74 & 0.69 & 0.79 \\
30.00 & 3000.00 & 100.00 & 0.77 & 0.96 & 0.74 & 0.69 & 0.79 \\
\bottomrule
\end{tabular}
\end{tabl

In [61]:
# summary table
summ = s[s.param_hash.isin(['583a1edc', 'c45449fc'])]

In [62]:
summ = summ[['query_group', 'variant', 'bdd', 'coco','lvis','objectnet','average']]
summ = summ.rename({'reg_norm_lambda':'$\\lambda$',
                    'reg_query_lambda':'$\\lambda_q$',
                    'reg_data_lambda':'$\\lambda_D$',   
                    'rocchio_beta':'$\\beta$',
                   'rocchio_gamma':'$\\gamma$',
                   'lvis':'LVIS',
                    'objectnet':'ObjNet',
                    'average':'Avg.',
                    'bdd':'BDD',
                    'coco':'COCO',
                    'query_group':'query group',
                   }, axis=1)


In [63]:
summ = summ.assign(variant=summ.variant.map(lambda nm : {'multi_reg':'this work', 'rocchio_update':'Rocchio'}.get(nm,nm)))

In [64]:
str = (summ.style
               .hide(axis=0)
               .format(precision=2)
               .highlight_max(axis=0, subset=['BDD', 'COCO', 'LVIS', 'ObjNet', 'Avg.'], props='bfseries: ;')
               .to_latex(label=f'tab:hyperparam_{k}',hrules=True,
                         clines='all;index', caption=f'sensitivity to hyperparamters for method:{k}'))
print(str)

\begin{table}
\caption{sensitivity to hyperparamters for method:rocchio_update}
\label{tab:hyperparam_rocchio_update}
\begin{tabular}{llrrrrr}
\toprule
query group & variant & BDD & COCO & LVIS & ObjNet & Avg. \\
\midrule
all queries & this work & \bfseries 0.79 & 0.96 & \bfseries 0.76 & \bfseries 0.70 & \bfseries 0.80 \\
all queries & Rocchio & 0.75 & \bfseries 0.97 & 0.74 & 0.70 & 0.79 \\
hard subset & this work & 0.24 & 0.75 & 0.44 & 0.40 & 0.46 \\
hard subset & Rocchio & 0.09 & 0.86 & 0.40 & 0.38 & 0.43 \\
\bottomrule
\end{tabular}
\end{table}

